In [2]:
import sqlite3

In [3]:
db_path = "/Users/jjpr/dev/mkgu/lookup.db"

In [4]:
conn = sqlite3.connect(db_path)

In [5]:
conn

In [6]:
cursor = conn.cursor()

In [7]:
cursor

In [8]:
sql_create_store = """CREATE TABLE store 
    (id integer primary key, type text, location text);"""

In [9]:
sql_create_assembly = """CREATE TABLE assembly 
    (id integer primary key, name text unique);"""

In [10]:
sql_create_assembly_store = """CREATE TABLE assembly_store 
    (id integer primary key, role text, 
    store_id integer, assembly_id integer);"""

In [ ]:
sql_last = "SELECT last_insert_rowid();"

In [11]:
cursor.execute(sql_create_store)

In [12]:
cursor.execute(sql_create_assembly)

In [13]:
cursor.execute(sql_create_assembly_store)

In [16]:
sql_insert_store = """INSERT INTO store 
(type, location) 
VALUES 
("S3", 
"https://s3.amazonaws.com/mkgu-dicarlolab-hvm/hvm_neuronal_features.nc")
"""

In [17]:
cursor.execute(sql_insert_store)
store_id = cursor.lastrowid
store_id

1

In [18]:
sql_insert_assy = """INSERT INTO assembly 
(name) 
VALUES 
("HvM")
"""

In [19]:
cursor.execute(sql_insert_assy)
assy_id = cursor.lastrowid
assy_id

1

In [20]:
sql_insert_assy_store = """INSERT INTO assembly_store 
(role, store_id, assembly_id) 
VALUES 
("HvM", ?, ?)
"""

In [21]:
cursor.execute(sql_insert_assy_store, (store_id, assy_id))
assy_store_id = cursor.lastrowid
assy_store_id

1

In [22]:
conn.commit()

In [8]:
conn.close()

In [9]:
conn

In [15]:
conn = sqlite3.connect(db_path)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [22]:
sql_lookup_assy = """SELECT 
    a.id as a_id, a.name 
    FROM
    assembly a
    WHERE
    a.name = ?
    """

In [20]:
sql_get_assy = """SELECT 
    a.id as a_id, a.name, a_s.id as a_s_id, a_s.role, 
    s.id as s_id, s.type, s.location 
    FROM
    assembly a
    JOIN assembly_store a_s ON a.id = a_s.assembly_id
    JOIN store s ON a_s.store_id = s.id
    WHERE
    a.name = ?
    """

In [27]:
cursor.execute(sql_lookup_assy, ("HvM",))
assy_result = cursor.fetchone()
{k: assy_result[k] for k in assy_result.keys()}

{'a_id': 1, 'name': 'HvM'}

In [28]:
cursor.execute(sql_get_assy, ("HvM",))
assy_store_result = cursor.fetchall()
{k: r[k] for r in assy_store_result for k in r.keys()}

{'a_id': 1,
 'a_s_id': 1,
 'location': 'https://s3.amazonaws.com/mkgu-dicarlolab-hvm/hvm_neuronal_features.nc',
 'name': 'HvM',
 'role': 'HvM',
 's_id': 1,
 'type': 'S3'}

In [30]:
class AssemblyRecord(object):
    """An AssemblyRecord stores information about the canonical location where the data
    for a DataAssembly is stored.  """
    def __init__(self, db_id, name, stores={}):
        self.db_id = db_id
        self.name = name
        self.stores = stores


class AssemblyStoreMap(object):
    """An AssemblyStoreMap links an AssemblyRecord to a Store.  """
    def __init__(self, db_id, role, store, assembly_record):
        self.db_id = db_id
        self.role = role
        self.store = store
        self.assembly_record = assembly_record


class Store(object):
    """A Store stores the location of a DataAssembly data file.  """
    def __init__(self, db_id, type, location, assemblies=[]):
        self.db_id = db_id
        self.type = type
        self.location = location
        self.assemblies = assemblies

In [31]:
assy = AssemblyRecord(assy_result["a_id"], assy_result["name"])
for r in assy_store_result:
    s = Store(r["s_id"], r["type"], r["location"], [assy])
    role = r["role"]
    a_s = AssemblyStoreMap(r["a_s_id"], role, s, assy)
    assy.stores[role] = a_s
assy

In [35]:
assy.name

'HvM'

In [36]:
[(s.role, s.store.type, s.store.location) for s in assy.stores.values()]

[('HvM',
  'S3',
  'https://s3.amazonaws.com/mkgu-dicarlolab-hvm/hvm_neuronal_features.nc')]